In [1]:
import torch
import os
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

from torch.quasirandom import SobolEngine

from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.kernels import MaternKernel, ScaleKernel
from gpytorch.constraints import Interval

from botorch import fit_gpytorch_mll
from botorch.utils.transforms import unnormalize, normalize
from botorch.models import SingleTaskGP
from botorch.models.transforms import Standardize
from botorch.optim import optimize_acqf
from botorch.acquisition import qLogExpectedImprovement
from botorch.sampling.stochastic_samplers import StochasticSampler

from helpers import (load_experiment_metadata, 
                    compute_nrmse_counts_all_edges, 
                    parse_loop_data_xml_to_pandas, 
                    create_taz_xml,
                    simulate_od,
                    od_xml_to_df,
                    xml2df_str)


In [2]:
!Pwd

/Users/chois/Gitsrcs/origin_destination_bayes_opt/bayesian_optimization


In [3]:
# base_path = "/home/bench/Gitsrcs/origin_destination_bayes_opt"
# base_path = "/Users/osorio/HEC/Research/Group/FacultyCollaborations/SeongjinChoi_UMN/Code_BO/origin_destination_bayes_opt-main"
base_path = "/Users/chois/Gitsrcs/origin_destination_bayes_opt"
os.chdir(base_path)

In [4]:
config_path = Path(base_path , 'config')
print(f"config_path: {config_path}")

config, sim_setup = load_experiment_metadata(config_path)

network_name = sim_setup['network_name']
model_name = sim_setup['model_name']

network_path = Path("network" , network_name)
taz2edge_xml = Path(base_path, network_path, 'taz.xml')
net_xml = Path(base_path, network_path, 'net.xml')
fixed_routes = Path(base_path, network_path, 'routes.csv')
file_gt_od = Path(base_path, network_path, 'od.xml')
additional_xml = Path(base_path, network_path, 'additional.xml')

out_path = f"output/{network_name}_{model_name}" 
Path(out_path).mkdir(parents=True, exist_ok=True)

gt_version_str = network_name           ## TODO : need to check if this is correct

EDGE_OUT_STR = f'edge_data_{network_name}.xml'
# suffix of simulation output edge file
TRIPS2ODS_OUT_STR = 'trips.xml'
SUMO_PATH = config["SUMO"]

sim_start_time = sim_setup['sim_start_time']
sim_end_time = sim_setup['sim_end_time']
sim_stat_freq_sec = sim_setup['sim_stat_freq_sec']
od_duration_sec = sim_setup['od_duration_sec']

n_init_search = sim_setup['n_init_search']

NITER = sim_setup["BO_niter"]
BATCH_SIZE = sim_setup["BO_batch_size"]
NUM_RESTARTS = sim_setup["BO_num_restarts"]
RAW_SAMPLES = sim_setup["BO_raw_samples"] 



config_path: /Users/chois/Gitsrcs/origin_destination_bayes_opt/config


In [5]:

# # taz2edge_xml = 'taz_new.xml'
# # net_xml = 'SFO.net.xml'
# # fixed_routes_xml = f'{base_path}/5hr_route_choice_set.csv'
# # od_duration_seconds = 5*60 

# # # duration of sample time for simulation output statistics
# # simulation_stat_freq_sec = od_duration_seconds
# # sim_end_time = od_duration_seconds
# # additional_xml = f'additional.add_statfreq{od_duration_seconds}.xml'

# # # suffix of simulation output edge file
# # EDGE_OUT_STR = 'edge_data_SFO.xml'
# # TRIPS2ODS_OUT_STR = 'trips.xml'
# # SUMO_PATH = '/usr/local/opt/sumo/share/sumo'

# od_duration_seconds = 30*60 

# # duration of sample time for simulation output statistics
# simulation_stat_freq_sec = od_duration_seconds
# sim_end_time = od_duration_seconds

# # TODO: it might be cleaner to replace this with a config file, i attached to my email an example. and one can define one config file per network. 
# network_name = "quickstart"
# model_name = "bo_vanilla"

# network_path = f"network/{network_name}"
# taz2edge_xml = f"{base_path}/{network_path}/taz.xml"
# net_xml = f"{base_path}/{network_path}/net.xml"
# fixed_routes = f"{base_path}/{network_path}/routes.csv"
# # od_xml = f"{network_path}/od.xml"       ## TODO : need to check if this is correct
# file_gt_od = f"{base_path}/{network_path}/od.xml"      ## TODO : need to check if this is correct
# # file_gt_edges                         ## TODO : need to check if this is necessary (not being used below)
# additional_xml = f"{base_path}/{network_path}/additional.xml"
# out_path = f"output/{network_name}_{model_name}"
# out_path = f"output/{network_name}_{model_name}"       ## TODO : need to check if this is correct
# # prefix_output = f"{out_path}/out"     ## TODO : need to check if this is correct
# gt_version_str = network_name           ## TODO : need to check if this is correct

# EDGE_OUT_STR = f'edge_data_{network_name}.xml'
# # suffix of simulation output edge file
# TRIPS2ODS_OUT_STR = 'trips.xml'
# # TODO I changed this path for it to work for me.
# SUMO_PATH = '/opt/homebrew/opt/sumo/share/sumo'
# #SUMO_PATH = "/usr/share/sumo"

# Path(out_path).mkdir(parents=True, exist_ok=True)




In [6]:
gt_version_str = 'v4'

# gt v4:
mean_od_val = 100
num_ods = 10

print('if you want to optimize them all (~86k) set num_ods as defined in commented line below')
#num_ods = routes_df.shape[0]

if you want to optimize them all (~86k) set num_ods as defined in commented line below


In [7]:
# od_xml = f'gt_od_{gt_version_str}.xml'
# file_gt = f'{base_path}/gt_od_{gt_version_str}.xml'
# file_gt_edges = f'{base_path}/gt_edges_{gt_version_str}.csv'
# prefix_output_gt = f'gt_{gt_version_str}'

In [8]:
# Get GT OD
print("Reading:",file_gt_od)
tree = ET.parse(file_gt_od)
root = tree.getroot()
gt_od_df =  xml2df_str(root, 'tazRelation')

gt_od_df.head()

Reading: /Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/od.xml


,from,to,count,data
0,taz91,taz93,500,None
1,taz91,taz94,500,None
2,taz92,taz93,900,None
3,taz92,taz94,900,None


In [9]:
print("Reading:",fixed_routes)
routes_df = pd.read_csv(fixed_routes, index_col=0)

Reading: /Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/routes.csv


In [10]:
gt_od_df = od_xml_to_df(file_gt_od)

total GT demand:  2800.0


In [11]:
gt_od_df.columns

Index(['from', 'to', 'count', 'data'], dtype='object')

## Vanilla BO


### Declare parameter space


In [12]:
# TODO: let's put all import  statements at the top of the notebook

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
dtype = torch.double

### Declare search space
# dimensionality of input space

dim_od = gt_od_df.shape[0]

#bounds = torch.tensor([
#    [ gt_od_df['count'].astype(float).min() - 2 for _ in range(dim_od)],
#    [ gt_od_df['count'].astype(float).max() + 2 for _ in range(dim_od)]
#], device=device, dtype=dtype) 

bounds = torch.tensor([
    [ 0 for _ in range(dim_od)],
    [ 2000 for _ in range(dim_od)]
], device=device, dtype=dtype) 


bounds



cpu


tensor([[   0.,    0.,    0.,    0.],
        [2000., 2000., 2000., 2000.]], dtype=torch.float64)

Run GT simulation


In [13]:
simulation_run_path =f'{out_path}'
simulation_gt_run_path =f'{out_path}/ground_truth'
Path(simulation_gt_run_path).mkdir(parents=True, exist_ok=True)
prefix_output_gt = f'{simulation_gt_run_path}/sim'

sim_edge_out_gt = f'{prefix_output_gt}_{EDGE_OUT_STR}'
new_od_xml = f'{simulation_gt_run_path}/od.xml'

base_od = gt_od_df.copy()
gt_od_vals = gt_od_df['count'].astype(float).to_numpy()
curr_od = gt_od_vals.copy()
base_od['count'] = curr_od
base_od = base_od.rename(columns={'fromTaz':'from', 'toTaz':'to'})        
create_taz_xml(new_od_xml, base_od, od_duration_sec, base_path)

print(base_od)

# Run simulation

simulate_od(new_od_xml, 
            prefix_output_gt, 
            base_path, 
            net_xml, 
            taz2edge_xml, 
            additional_xml,
            routes_df,
            sim_end_time,
            TRIPS2ODS_OUT_STR)



Created  /Users/chois/Gitsrcs/origin_destination_bayes_opt/output/quickstart_bo_vanilla/ground_truth/od.xml
    from     to  count  data
0  taz91  taz93  500.0  None
1  taz91  taz94  500.0  None
2  taz92  taz93  900.0  None
3  taz92  taz94  900.0  None
od2trips  --spread.uniform --taz-files /Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/taz.xml --tazrelation-files output/quickstart_bo_vanilla/ground_truth/od.xml -o /Users/chois/Gitsrcs/origin_destination_bayes_opt/output/quickstart_bo_vanilla/ground_truth/sim_trips_beforeRteUpdates.xml
Success.time 299.17
###### Running SUMO #######
Seed 0
sumo --output-prefix output/quickstart_bo_vanilla/ground_truth/sim_ --ignore-route-errors=true --net-file=/Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/net.xml --routes=/Users/chois/Gitsrcs/origin_destination_bayes_opt/output/quickstart_bo_vanilla/ground_truth/sim_trips.xml -b 0 -e 57600 --additional-files /Users/chois/Gitsrcs/origin_destination_bayes_opt/

Loading additional-files from '/Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/additional.xml' ... done (2ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
Step #57600.00 (0ms ?*RT. ?UPS, vehicles TOT 2800 ACT 0 BUF 0)                            
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 0.69s
 Real time factor: 83720.9
 UPS: 1698090.116279
Vehicles: 
 Inserted: 2800
 Running: 0
 Waiting: 0
Statistics (avg of 2800):
 RouteLength: 4459.95
 Speed: 10.71
 Duration: 417.25
 WaitingTime: 0.36
 TimeLoss: 91.33
 DepartDelay: 693.70

DijkstraRouter answered 5435 queries and explored 10.94 edges on average.
DijkstraRouter spent 0.00s answering queries (0.00ms on average).


Read output of GT simulation


In [14]:
df_edge_gt, _, _ = parse_loop_data_xml_to_pandas(base_path, sim_edge_out_gt, prefix_output_gt,SUMO_PATH)
# picking at edges as GT edges
num_gt_edges = df_edge_gt.shape[0]
print("Number of GT edges:",num_gt_edges)
gt_edge_data = df_edge_gt\
    .sort_values(by=['interval_nVehContrib'], ascending=False)\
    .iloc[:num_gt_edges]

# gt_edge_data.shape


Number of GT edges: 14


In [15]:
gt_edge_data

,edge_id,interval_nVehContrib,interval_harmonicMeanSpeed
1,D4,1800.0,8.527143
6,L14,1800.0,11.386250
11,L4,1800.0,9.915000
2,D5,1400.0,11.488750
3,D7,1400.0,11.294444
4,L10,1400.0,11.441250
9,L18,1400.0,11.635000
12,L5,1400.0,11.353750
13,L7,1400.0,11.046667
0,D2,1000.0,8.445000


In [16]:
bounds.shape

torch.Size([2, 4])

In [17]:
# full grid search
n_full_search = 21
candidates = []

# print(dim_od)
for i in range(dim_od):
    candidates.append(torch.linspace(0,1,n_full_search))

search_space = torch.meshgrid(candidates)
search_space = torch.stack(search_space , 0)
search_space.shape
search_space = search_space.view(dim_od, -1)
search_space = search_space.transpose(0,1)
print(f"search_space shape = {search_space.shape}")

# map the normalized into the original parameter space
train_X0 = unnormalize(search_space, bounds)
train_X0 = train_X0[1:,:]
train_X0

search_space shape = torch.Size([625, 4])


/Users/chois/Gitsrcs/origin_destination_bayes_opt/.venv/lib/python3.9/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


tensor([[   0.,    0.,    0.,  500.],
        [   0.,    0.,    0., 1000.],
        [   0.,    0.,    0., 1500.],
        ...,
        [2000., 2000., 2000., 1000.],
        [2000., 2000., 2000., 1500.],
        [2000., 2000., 2000., 2000.]], dtype=torch.float64)

In [18]:
#num_epsilon_iter = 2
ods_epsilon = []
loss_all = []
batch_data_i = []

# Base OD which we will update their count entries
base_od = gt_od_df.copy()
gt_od_vals = gt_od_df['count'].astype(float).to_numpy()

for i in tqdm(range(train_X0.shape[0])):
      x = train_X0[i]
#for i , x in enumerate(
#      [[ 94.66438596,  91.97375804, 101.82277249, 112.44778006,
#            105.33019264,  92.62166575,  99.8673423 ,  93.71928772,
#            116.16658554,  94.79717515],
#      [ 97.4, 114.9, 104.1, 100. , 109.1, 106.7,  87.8, 101.1, 113.9,109.4]]):
      print(f"########### OD: {i} ###########")
      print(x)
      
      Path(f'{simulation_run_path}/full_search').mkdir(parents=True, exist_ok=True)
      new_od_xml = f'{simulation_run_path}/full_search/gt_od_{gt_version_str}_{i}.xml'
      prefix_output_init = f'{simulation_run_path}/full_search/fullsearch_{i}'

      # Generate OD
      #curr_od = gt_od_vals.copy()
      curr_od = np.array(x)

      print(f'total expected GT demand: {np.sum(curr_od)}')

      ###
      # create OD xml file 
      ###
      base_od['count'] = curr_od
      # round to 1 decimal point
      base_od['count'] = [round(elem, 1) for elem in base_od['count']]     
      base_od = base_od.rename(columns={'fromTaz':'from', 'toTaz':'to'})        
      create_taz_xml(new_od_xml, base_od, od_duration_sec, base_path)
      ods_epsilon.append(curr_od)

      # simulate gt od
      simulate_od(new_od_xml, 
                  prefix_output_init, 
                  base_path, 
                  net_xml, 
                  taz2edge_xml, 
                  additional_xml, 
                  routes_df,
                  sim_end_time,
                  TRIPS2ODS_OUT_STR)

      ## Compute loss
      #prefix_output = f'full_search/sobol_{i}'
      sim_edge_out = f'{base_path}/{prefix_output_init}_{EDGE_OUT_STR}'
      print(sim_edge_out)
      curr_loop_stats, _, _ = parse_loop_data_xml_to_pandas(base_path, sim_edge_out,prefix_output_init,SUMO_PATH)
      curr_loss = compute_nrmse_counts_all_edges(gt_edge_data, curr_loop_stats)

      loss_all.append(curr_loss)
      print(f"############## loss: {curr_loss} ##############")

      # Parse training data
      df_curr = pd.DataFrame(curr_od.reshape(1,dim_od),
                        columns = [f"x_{i+1}" for i in range(dim_od)])
      df_curr['loss'] = curr_loss
      batch_data_i.append(df_curr)



  0%|          | 0/624 [00:00<?, ?it/s]

########### OD: 0 ###########
tensor([  0.,   0.,   0., 500.], dtype=torch.float64)
total expected GT demand: 500.0
Created  /Users/chois/Gitsrcs/origin_destination_bayes_opt/output/quickstart_bo_vanilla/full_search/gt_od_v4_0.xml
od2trips  --spread.uniform --taz-files /Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/taz.xml --tazrelation-files output/quickstart_bo_vanilla/full_search/gt_od_v4_0.xml -o /Users/chois/Gitsrcs/origin_destination_bayes_opt/output/quickstart_bo_vanilla/full_search/fullsearch_0_trips_beforeRteUpdates.xml
Success.time 299.10
###### Running SUMO #######
Seed 0
sumo --output-prefix output/quickstart_bo_vanilla/full_search/fullsearch_0_ --ignore-route-errors=true --net-file=/Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/net.xml --routes=/Users/chois/Gitsrcs/origin_destination_bayes_opt/output/quickstart_bo_vanilla/full_search/fullsearch_0_trips.xml -b 0 -e 57600 --additional-files /Users/chois/Gitsrcs/origin_destination_b

  0%|          | 1/624 [00:00<04:26,  2.34it/s]

Step #57600.00 (0ms ?*RT. ?UPS, vehicles TOT 500 ACT 0 BUF 0)                             
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 0.16s
 Real time factor: 366879
 UPS: 1599656.050955
Vehicles: 
 Inserted: 500
 Running: 0
 Waiting: 0
Statistics (avg of 500):
 RouteLength: 4948.60
 Speed: 9.88
 Duration: 502.29
 WaitingTime: 0.37
 TimeLoss: 139.17
 DepartDelay: 120.19

DijkstraRouter answered 827 queries and explored 12.84 edges on average.
DijkstraRouter spent 0.00s answering queries (0.00ms on average).
/Users/chois/Gitsrcs/origin_destination_bayes_opt/output/quickstart_bo_vanilla/full_search/fullsearch_0_edge_data_quickstart.xml
############## loss: 0.8423992624218193 ##############
########### OD: 1 ###########
tensor([   0.,    0.,    0., 1000.], dtype=torch.float64)
total expected GT demand: 1000.0
Created  /Users/chois/Gitsrcs/origin_destination_bayes_opt/output/quickstart_bo_vanilla/full_search/gt_od_v4_1.xm

Success.time 299.85
###### Running SUMO #######
Seed 0
sumo --output-prefix output/quickstart_bo_vanilla/full_search/fullsearch_1_ --ignore-route-errors=true --net-file=/Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/net.xml --routes=/Users/chois/Gitsrcs/origin_destination_bayes_opt/output/quickstart_bo_vanilla/full_search/fullsearch_1_trips.xml -b 0 -e 57600 --additional-files /Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/additional.xml --duration-log.statistics --xml-validation never --vehroutes routes.vehroutes.xml --verbose --no-warnings --mesosim true --seed 0
Loading net-file from '/Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/net.xml' ... done (2ms).
Loading additional-files from '/Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/additional.xml' ... done (1ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.


...

100%|█████████▉| 623/624 [20:33<00:04,  4.09s/it]

Step #57600.00 (0ms ?*RT. ?UPS, vehicles TOT 7500 ACT 0 BUF 0)                            
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 3.95s
 Real time factor: 14571.2
 UPS: 789233.746522
Vehicles: 
 Inserted: 7500
 Running: 0
 Waiting: 0
Statistics (avg of 7500):
 RouteLength: 4427.37
 Speed: 10.66
 Duration: 415.98
 WaitingTime: 0.35
 TimeLoss: 92.97
 DepartDelay: 1962.11

DijkstraRouter answered 14864 queries and explored 10.83 edges on average.
DijkstraRouter spent 0.01s answering queries (0.00ms on average).
/Users/chois/Gitsrcs/origin_destination_bayes_opt/output/quickstart_bo_vanilla/full_search/fullsearch_622_edge_data_quickstart.xml
############## loss: 1.7419055645394377 ##############
########### OD: 623 ###########
tensor([2000., 2000., 2000., 2000.], dtype=torch.float64)
total expected GT demand: 8000.0
Created  /Users/chois/Gitsrcs/origin_destination_bayes_opt/output/quickstart_bo_vanilla/full_search/gt_o

Success.time 299.33
###### Running SUMO #######
Seed 0
sumo --output-prefix output/quickstart_bo_vanilla/full_search/fullsearch_623_ --ignore-route-errors=true --net-file=/Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/net.xml --routes=/Users/chois/Gitsrcs/origin_destination_bayes_opt/output/quickstart_bo_vanilla/full_search/fullsearch_623_trips.xml -b 0 -e 57600 --additional-files /Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/additional.xml --duration-log.statistics --xml-validation never --vehroutes routes.vehroutes.xml --verbose --no-warnings --mesosim true --seed 0
Loading net-file from '/Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/net.xml' ... done (3ms).
Loading additional-files from '/Users/chois/Gitsrcs/origin_destination_bayes_opt/network/quickstart/additional.xml' ... done (1ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.


100%|██████████| 624/624 [20:38<00:00,  1.98s/it]

Step #57600.00 (0ms ?*RT. ?UPS, vehicles TOT 8000 ACT 0 BUF 0)                            
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 4.51s
 Real time factor: 12763.1
 UPS: 745643.252825
Vehicles: 
 Inserted: 8000
 Running: 0
 Waiting: 0
Statistics (avg of 8000):
 RouteLength: 4459.95
 Speed: 10.62
 Duration: 420.64
 WaitingTime: 0.38
 TimeLoss: 95.33
 DepartDelay: 2093.61

DijkstraRouter answered 15863 queries and explored 10.95 edges on average.
DijkstraRouter spent 0.02s answering queries (0.00ms on average).
/Users/chois/Gitsrcs/origin_destination_bayes_opt/output/quickstart_bo_vanilla/full_search/fullsearch_623_edge_data_quickstart.xml
############## loss: 1.901416476047427 ##############


In [19]:
df_initial_bo = pd.concat(batch_data_i)
df_initial_bo.head()

,x_1,x_2,x_3,x_4,loss
0,0.0,0.0,0.0,500.0,0.842399
0,0.0,0.0,0.0,1000.0,0.698386
0,0.0,0.0,0.0,1500.0,0.643256
0,0.0,0.0,0.0,2000.0,0.698386
0,0.0,0.0,500.0,0.0,0.858551


In [20]:
simulation_run_path

'output/quickstart_bo_vanilla'

In [21]:
# Save initial dataset
df_initial_bo.to_csv(f"{simulation_run_path}/full_search/data_set_ods_0_2000.csv",index=None)
